## Design the call function (basic)

Aim: given a batch of x=content, fill the properties by graph models.

In [1]:
import sys
for path in ['../','../../','../../../','../../../../']:
    sys.path.append(path+"CNT_Code/V1.05.22")
    
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from RegressionTools import *
from GraphTools import *
from GeneralTools import *
# from GraphModel import *
from GraphModel_cnt import *
from CNT_Dataset import *
from ModelSelection import *

In [2]:
data_org = pd.read_excel('data_all_back_A.xlsx') # 2020.06.06 use data back -- no log(*)
graph_array = data_load('graph_array_368.dat')

In [3]:
idx = ['P3HT vol (µL)','D1 vol (µL)', 'D2 vol (µL)', 'D6 vol (µL)', 'D8 vol (µL)',]
# idy = 'Conductivity (modified predict)'
idy = 'Y'; targy = 'Cond'
idz = 'R'; targz = 'Abs'

id_targ = idy; this_targ = targy
# id_targ = idz; this_targ = targz

In [4]:
gm_array = data_load('gm_array.dat')

In [5]:
def _call(x):
    res_array = []
    for i in range(len(x)):
        gm = np.random.choice(gm_array)
        data_x = pd.DataFrame(x[i:i+1],columns=idx)
        for s in ['A','R','S','T','Y']: data_x[s] = np.nan;
        res = gm.predict(data=data_x,n_predict=1,ensemble=False,random=True)
        res['predicted by graph'] = nx2str(gm.G)
        res_array.append(res)
    return pd.concat(res_array)

## re measure

In [6]:
data_orgs = []
for i in range(20):
#     data_orgs.append(pd.read_excel('./BO-history/Artificial_BO_20200214T%d_history_Cond.xlsx'%i, sheet_name='20'))
    data_orgs.append(pd.read_excel('BO-history/Artificial_BO_20200701_20T%d_history_Cond.xlsx'%(i), sheet_name='20'))
idx = ['P3HT vol (µL)','D1 vol (µL)', 'D2 vol (µL)', 'D6 vol (µL)', 'D8 vol (µL)',]

In [7]:
num_measure = 20
data_array = []
filename = 'BO-history-Cond-final.xlsx'

In [8]:
def fill_mean_std(data,num_measure=40):
    X = data[idx].values
    y_mean_std_array = []
    for i in range(len(X)):
        print("%d of %d"%(i+1,len(X)),end='\r')
        yi = []
        x = X[i:i+1]
        for i in range(num_measure):
            yi.append(_call(x)['Y'])
        
        y_mean_std_array.append(np.concatenate([[np.mean(yi),np.std(yi),np.median(yi)],np.percentile(yi,[25,75])]))
    y_mean_std_array = np.array(y_mean_std_array)
    data['Y-mean'] = y_mean_std_array[:,0]
    data['Y-std'] = y_mean_std_array[:,1]
    data['Y-median'] = y_mean_std_array[:,2]
    data['Y-25%'] = y_mean_std_array[:,3]
    data['Y-75%'] = y_mean_std_array[:,4]
    return data

In [9]:
def write_to_excel(data_array,filename='tmp.xlsx'):
    from openpyxl import load_workbook
    with pd.ExcelWriter(filename) as writer:
        for i,data in enumerate(data_array):
            data.to_excel(writer,sheet_name='Test%d'%(i+1))

In [10]:
len(data_orgs[0])*len(data_orgs)*20/(50*40)*3

120.0

In [15]:
# num_measure = 20
# data_array = []
# filename = 'BO-history-Cond-final-2.xlsx'
import time as tm
for i,data in enumerate(data_orgs):
    print("data %d: "%(i+1), tm.ctime())
    data = fill_mean_std(data)
    data_array.append(data)
    write_to_excel(data_array,filename)
print(tm.ctime())

data 1:  Thu Jul  2 00:03:44 2020
data 2:  Thu Jul  2 00:04:37 2020
data 3:  Thu Jul  2 00:05:29 2020
data 4:  Thu Jul  2 00:06:22 2020
data 5:  Thu Jul  2 00:07:15 2020
data 6:  Thu Jul  2 00:08:08 2020
data 7:  Thu Jul  2 00:09:01 2020
data 8:  Thu Jul  2 00:09:55 2020
data 9:  Thu Jul  2 00:10:48 2020
data 10:  Thu Jul  2 00:11:41 2020
data 11:  Thu Jul  2 00:12:35 2020
data 12:  Thu Jul  2 00:13:30 2020
data 13:  Thu Jul  2 00:14:23 2020
data 14:  Thu Jul  2 00:15:16 2020
data 15:  Thu Jul  2 00:16:09 2020
data 16:  Thu Jul  2 00:17:03 2020
data 17:  Thu Jul  2 00:17:57 2020
data 18:  Thu Jul  2 00:18:50 2020
data 19:  Thu Jul  2 00:19:44 2020
data 20:  Thu Jul  2 00:20:37 2020
Thu Jul  2 00:21:31 2020


In [16]:
filename

'BO-history-Cond-final.xlsx'